In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM

In [ ]:
train_feature_path = "./datasets/train/train_feature.npz"
valid_feature_path = "./datasets/valid/valid_feature.npz"
test_feature_path = "./datasets/test/test_feature.npz"

In [ ]:
data = np.load(train_feature_path, allow_pickle=True)
valid_data = np.load(valid_feature_path, allow_pickle=True)
test_data = np.load(test_feature_path, allow_pickle=True)

train_deep_X = data['features']
train_deep_Y = data['label']

valid_deep_X = valid_data['features']
valid_deep_Y = valid_data['label']


train_X_deep_flattened = train_deep_X.reshape(train_deep_X.shape[0], -1)
valid_X_deep_flattened = valid_deep_X.reshape(valid_deep_X.shape[0], -1)
print(train_X_deep_flattened.shape)

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Define the number of components you want to reduce to
n_components = 500
pca = PCA(n_components=n_components)
train_embeddings_pca = pca.fit_transform(train_X_deep_flattened)
validation_embeddings_pca = pca.transform(valid_X_deep_flattened)

explained_variance_ratio = np.sum(pca.explained_variance_ratio_)
print(f"Total retained variance by {n_components} components: {explained_variance_ratio * 100:.2f}%")

print(f"Train Embeddings Shape after PCA: {train_embeddings_pca.shape}")
print(f"Validation Embeddings Shape after PCA: {validation_embeddings_pca.shape}")

In [ ]:
# Dictionary of models
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Ridge Regression":RidgeClassifier(),
#     "Linear Discriminant" : LinearDiscriminantAnalysis(),
    "XG Boost" : XGBClassifier(),
#     "Naive Bayes" : GaussianNB(priors=None, var_smoothing=1.0),
    "SVM": SVC(C= 0.01, degree=3, gamma='scale', kernel= 'linear'),
}

percentages = np.linspace(0.2, 1.0, 5)
results = []
for percent in percentages:
    n_samples = int(len(train_embeddings_pca) * percent)
    print(f"\nTraining with {percent*100:.1f}% of the data ({n_samples} samples)")

    # Select a subset of training data based on the percentage
    X_subset = train_embeddings_pca[:n_samples]
    y_subset = train_deep_Y[:n_samples]

    # Train and evaluate models
    for model_name, model in models.items():
        model.fit(X_subset, y_subset)

        y_pred_valid = model.predict(validation_embeddings_pca)
        validation_accuracy = accuracy_score(valid_deep_Y, y_pred_valid)
        print(f"Validation accuracy for {model_name}: {validation_accuracy}")
        results.append([percent * 100, model_name, validation_accuracy])


df = pd.DataFrame(results, columns=['percent', 'model_name', 'validation_accuracy'])

plt.figure(figsize=(10, 6))

for model_name in df['model_name'].unique():
    model_df = df[df['model_name'] == model_name]
    plt.plot(model_df['percent'], model_df['validation_accuracy'], marker='o', label=model_name)

plt.title('Validation Accuracy vs. Percentage of Data Used (With Hyperparameter Tuning)')
plt.xlabel('Percentage of Data Used (%)')
plt.ylabel('Validation Accuracy')
plt.legend(title='Model Name')
plt.show()
